In [ ]:
/home/halechr/repos/PhoGlobusHelpers/filelists/GreatGDriveMigration2023/dirsize

# 2023-09-12 - New Filelist Transfer

In [8]:
from typing import List
from pathlib import Path
import numpy as np
import pandas as pd

def convert_filelist_to_new_parent(filelist_source: List[Path], original_parent_path: Path = Path(r'/media/MAX/cloud/turbo/Data'), dest_parent_path: Path = Path(r'/media/MAX/Data')):
    """ Converts a list of file paths from their current parent, specified by `original_parent_path`, to their new parent `dest_parent_path` """
    filelist_dest = []
    for path in filelist_source:
        relative_path = str(path.relative_to(original_parent_path))
        new_path = Path(dest_parent_path) / relative_path
        filelist_dest.append(new_path)
    return filelist_dest

# Load filelist from disk
# active_filelist_path = Path('/home/halechr/repo/PhoGlobusHelpers/filelists/session_results_filelist_2023-07-12.csv').resolve()
active_filelist_path = Path('/home/halechr/repos/PhoGlobusHelpers/filelists/GreatGDriveMigration2023/dirsize/Bapun_dirsize.csv').resolve()

filelist_df = pd.read_csv(active_filelist_path, header=0, names=["name", "modified_dt", "size_bytes"])
filelist_df


,name,modified_dt,size_bytes
0,Hiro_10hrPost_reclustered/wake10hrPost-behavio...,2018-01-04 15:20:20,48898
1,Hiro_10hrPost_reclustered/wake10hrPost-basics.mat,2018-01-04 15:20:18,2260
2,RatJ/timestamps.npy,2020-01-19 17:46:39,13857681728
3,ReclusteredDatas/Hiro/readme.txt,2018-04-13 01:08:57,556
4,RatK/Day4Openfield/RatK_Day4_2019-08-16_04-42-...,2022-08-03 14:51:21,86624914
...,...,...,...
5607,RatV/RatVDay2SD/Raw_data/2021-10-04_08-15-13/R...,2021-10-04 09:02:23,128
5608,RatV/RatVDay2SD/Raw_data/2021-10-04_08-15-13/R...,2021-10-04 09:02:23,128
5609,RatV/RatVDay2SD/Raw_data/2021-10-04_08-15-13/R...,2021-10-04 09:02:23,128
5610,RatV/RatVDay2SD/Raw_data/2021-10-04_08-15-13/R...,2021-10-04 09:02:23,128


In [9]:
np.sum(filelist_df['size_bytes']/(1024.0 * 1024.0)) # Convert to GigaBytes (GB)



21955757.33845806

In [6]:
filelist_df.dtypes

name           object
modified_dt    object
size_bytes      int64
dtype: object

In [ ]:
filelist_source = [Path(a_path_str).resolve() for a_path_str in filelist_df.Path]



source_parent_path = Path(r'/media/MAX/cloud/turbo/Data')
dest_parent_path = Path(r'/media/MAX/Data')
# # Build the destination filelist from the source_filelist and the two paths:
filelist_dest = convert_filelist_to_new_parent(filelist_source, original_parent_path=source_parent_path, dest_parent_path=dest_parent_path)
filelist_dest

# filelist_source
# filelist_dest